Keypoints

- 모델 입력은 항상 (batch_size, sequence_length)
- 길이가 다르면 → padding 필수
- padding이 있으면 → attention_mask 필수
- tokenizer를 쓰면 이 모든 걸 자동 처리
- 수동 구현은 이해용, 실무는 tokenizer에 맡김

In [2]:
# Model & Tokenizer Load
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
sentence = "I've been waiting for a HuggingFace course my whole life."

inputs = tokenizer(sentence, return_tensors="pt")
outputs = model(**inputs)

print(outputs.logits)

tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>)


## Multiple Sentences

In [ ]:
sentences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]

inputs = tokenizer(sentences, return_tensors="pt", padding=True) # Automatically convert to batch by return_tensors='pt' (padding, truncation) + attention mask creation
outputs = model(**inputs)

print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [11]:
print("input_ids:\n", inputs["input_ids"])
print("attention_mask:\n", inputs["attention_mask"])

input_ids:
 tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]])
attention_mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])


## Long Sequence (Truncation)

In [13]:
long_sentence = "Hello " * 1000

inputs = tokenizer(
    long_sentence,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

print(inputs["input_ids"].shape)


torch.Size([1, 512])
